In [ ]:
import pandas as pd
import json
from nltk.tokenize import word_tokenize
import re
import cohere
import time

In [ ]:
# =============================================================================
# SEZIONE 1: Caricamento topic representations
# -----------------------------------------------------------------------------

df = pd.read_csv("./Results/topics_overview.tsv", sep="\t") 


In [ ]:
# =============================================================================
# SEZIONE 2: Generazione etichette dei topic con LLM (Cohere)
# -----------------------------------------------------------------------------
# Questa sezione del notebook:
# 1) Inizializza il client Cohere con la propria API key.
# 2) Definisce un messaggio di sistema che istruisce il modello a produrre
#    un’etichetta breve (3–4 parole), in italiano, senza formattazioni o spiegazioni.
# 3) Scorre tutte le righe del DataFrame `df` ed estrae, per ciascuna riga,
#    le parole rappresentative del topic dalla colonna `Representation`.
# 4) Costruisce il prompt utente (“Words: ...\nLabel:”) e invia la richiesta
#    al modello `command-a-03-2025` tramite `co.chat(...)`.
# 5) Legge la risposta del modello, stampa la label e la aggiunge alla lista `labels`
#    (eventualmente da assegnare poi a `df['Label']`).
# 6) Inserisce una pausa di 60 secondi ogni 10 richieste (`time.sleep(60)`)
#    per rispettare i limiti di frequenza dell’API (rate limiting).
# =============================================================================


co = cohere.ClientV2(api_key="") # inserire la propria API key di cohere qui, vedi https://cohere.com/ (API gratuita)

# prompt da passare all'LLM. In inglese per cohere perchè il modello funziona meglio in inglese, ma si può modificare il prompt a piacere (anche in italiano)
system_message = f"""We perform topic modelling on a set of actions that an italian public administration might want to do.  
        The following words are all representative of the actions grouped in the same topic, the words are in decrescent order of relevance.
        You should find a label that describes the type of actions that this topic represents, in a short label (3 or 4 words max). Prefer substantives.
        Try to be very specific in your label. Only return one potential name in Italian. Output only the label. 
        Do not repeat the prompt. Do not use boldface. Do not add example or exaplanations. Only the label\n"""

index = 0 
labels = []
for i in range(len(df)):
    index = index + 1
    words = df['Representation'].iloc[i]
    prompt = f"Words: {words}\nLabel:"
    res = co.chat(
        model="command-a-03-2025",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},  

        ],
    )
    print(res.message.content[0].text)
    labels.append(res.message.content[0].text)

    if index %10==0: # ogni 10 richieste, pausa di 60 secondi per rispettare i limiti di frequenza dell'API
        time.sleep(60)


In [ ]:
# =============================================================================
# SEZIONE 3: Salvataggio risultati
# -----------------------------------------------------------------------------
# Questa sezione del notebook:
# 1) Aggiunge la colonna label al dataframe dei topic.
# 2) Esporta il dataframe aggiornato in un file tsv.
# =============================================================================

df['Label'] = labels
df.to_csv("./Results/topics_overview.tsv", sep="\t", index=False)